In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
train_df = pd.read_excel("训练.xlsx")
test1_df = pd.read_excel("测试A.xlsx")
test2_df = pd.read_excel("测试B.xlsx")

In [3]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(500,)

In [4]:
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [5]:
new_column = test1_df.columns
train_df = train_df[new_column]

In [6]:
#trainx = pd.concat([train_df, test1_df, test2_df], axis=0)
trainx = pd.concat([train_df, test1_df], axis=0)
trainx.shape

(600, 8027)

In [7]:
# ints or strings
names = trainx.columns
dfs = trainx.dtypes
ints = []
for i in range(len(names)):
    if 'int' in str(dfs[i]) or 'object' in str(dfs[i]):
        ints.append(names[i])
print(len(ints))

1544


In [8]:
remains1 = []
remains2 = []
tobedrop = []
for n in ints:
    lu1 = len(train_df[n].unique())
    lu2 = len(trainx[n].unique())
    if lu1 < 10 and lu1 > 1:
        remains1.append(n)
    if lu2 < 10 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
print(len(remains1), len(remains2), len(tobedrop))

367 361 504


In [9]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

324


In [10]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.9:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [11]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(train_df[n]):
        lm1.append(n)
    if LastRemainJudge(trainx[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

236 241 235


In [12]:
names = trainx.columns
dts = trainx.dtypes
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = trainx[dfx_float_n]
dfx_onehot = trainx[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

235
6483
(600, 6483) (600, 235)


In [13]:
drop_names = []
for n in dfx_float_n:
    x = np.sum(dfx_float[n].isnull())
    if x > 300:
        drop_names.append(n)
print(len(drop_names))

198


In [16]:
dfx_float.drop(drop_names, axis=1, inplace=True)
print(dfx_float.shape)

(600, 6009)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

2513 2498
0


In [18]:
# not move sames first, fill nan
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

832


In [19]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))

3493


In [20]:
dfx_float.drop(drop_names, axis=1, inplace=True)
print(dfx_float.shape)

(600, 2516)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
names = dfx_float.columns
losts = []
for n in names:
    d = dfx_float[n]
    x = np.sum(d.isnull())
    if x > 0:
        print(n, x)
        losts.append(n)
print(len(losts))

210X213 1
210X215 1
220X26 1
220X27 1
220X28 1
220X29 1
220X30 1
220X31 1
220X32 1
220X33 1
220X53 1
220X54 1
220X55 1
220X66 1
220X67 1
220X75 1
220X83 1
220X91 1
220X107 1
220X108 1
220X223 1
220X224 1
220X225 4
220X227 4
220X228 4
220X229 4
220X230 4
220X231 4
220X233 4
220X235 4
220X241 4
220X247 4
220X248 4
220X249 4
220X260 4
220X262 4
220X266 4
220X269 4
220X270 4
220X271 4
220X272 4
220X273 4
220X274 4
220X275 4
220X277 4
220X279 4
220X281 4
220X282 4
220X284 4
220X286 4
220X288 4
220X289 4
220X290 4
220X291 4
220X293 4
220X294 4
220X295 4
220X296 4
220X297 4
220X298 4
220X299 4
220X300 4
220X301 4
220X302 4
220X303 4
220X304 4
220X305 4
220X306 4
220X307 4
220X308 4
220X309 4
220X310 4
220X311 4
220X312 4
220X313 4
220X314 4
220X315 4
220X316 4
220X317 4
220X318 4
220X321 4
220X322 4
220X323 4
220X324 4
220X325 4
220X326 4
220X327 4
220X328 4
220X329 4
220X330 4
220X331 4
220X332 4
220X333 4
220X334 4
220X335 4
220X336 4
220X337 4
220X338 4
220X339 4
220X340 4
220X341 4
220X34

In [21]:
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    assert tmps>0
    dfx_float[n][c.isnull()] = tmpm


/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
print(np.sum(dfx_float.isnull().values))

0


In [23]:
mid = dfx_float-np.min(dfx_float)+1
dfx_log1p = np.log1p(mid)
del mid
print(np.sum(dfx_log1p.isnull().values))

0


In [24]:
names = dfx_float.columns
for n in names:
    m1 = np.mean(dfx_float[n])
    s1 = np.std(dfx_float[n])
    assert s1 > 0
    dfx_float[n] = (dfx_float[n]-m1)/s1
    m2 = np.mean(dfx_log1p[n])
    s2 = np.std(dfx_log1p[n])
    assert s2 > 0
    dfx_log1p[n] = (dfx_log1p[n]-m2)/s2

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
# make onehots onehot
ohnames = dfx_onehot.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfx_onehot[n], prefix='onehot_'+str(i))
    dfx_onehot = pd.concat([dfx_onehot, tmp], axis=1)
    print(dfx_onehot.shape)

(600, 237)
(600, 240)
(600, 242)
(600, 244)
(600, 246)
(600, 253)
(600, 260)
(600, 262)
(600, 265)
(600, 272)
(600, 275)
(600, 281)
(600, 284)
(600, 293)
(600, 296)
(600, 298)
(600, 301)
(600, 304)
(600, 307)
(600, 311)
(600, 317)
(600, 320)
(600, 324)
(600, 328)
(600, 331)
(600, 335)
(600, 338)
(600, 342)
(600, 347)
(600, 351)
(600, 355)
(600, 358)
(600, 360)
(600, 365)
(600, 369)
(600, 374)
(600, 377)
(600, 380)
(600, 384)
(600, 390)
(600, 399)
(600, 401)
(600, 403)
(600, 405)
(600, 409)
(600, 411)
(600, 413)
(600, 418)
(600, 423)
(600, 430)
(600, 432)
(600, 441)
(600, 450)
(600, 453)
(600, 459)
(600, 461)
(600, 467)
(600, 473)
(600, 477)
(600, 479)
(600, 482)
(600, 484)
(600, 487)
(600, 496)
(600, 500)
(600, 504)
(600, 507)
(600, 509)
(600, 511)
(600, 513)
(600, 517)
(600, 525)
(600, 531)
(600, 533)
(600, 538)
(600, 543)
(600, 549)
(600, 553)
(600, 560)
(600, 567)
(600, 571)
(600, 574)
(600, 578)
(600, 583)
(600, 586)
(600, 592)
(600, 594)
(600, 596)
(600, 601)
(600, 607)
(600, 609)

In [26]:
dfx_onehot.drop(ohnames, axis=1, inplace=True)
print(dfx_onehot.shape)

(600, 985)


In [27]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [60]:
X_new = SelectKBest(f_regression, k=500).fit_transform(dfx_log1p[:500], train_y)
print(X_new.shape)

(500, 300)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:292: RuntimeWarning: invalid value encountered in sqrt
  n_samples * X_means ** 2)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:303: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/home/magnusterra/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/magnusterra/anaconda3/

In [28]:
dfx_log1p.shape

(600, 2516)

In [29]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
names = dfx_float.columns
lr = LinearRegression()
lr.fit(dfx_float[:500], train_y)
fn = names[lr.coef_>0]
lr = LinearRegression()
lr.fit(dfx_log1p[:500], train_y)
ln = names[lr.coef_>0]

In [32]:
print(len(fn), len(ln))
print(len(set(fn) & set(ln)))

1269 1266
1082


In [33]:
xfloatlr = dfx_float[fn]
xlog1plr = dfx_log1p[ln]

In [34]:
lr = LinearRegression()
lr.fit(dfx_onehot[:500], train_y)
print(np.sum(lr.coef_>0))
names = dfx_onehot.columns
xonehot = dfx_onehot[names[lr.coef_>0]]
print(xonehot.shape)

472
(600, 472)


In [35]:
def MSE(y_raw, y_pred):
    print(np.mean(np.square(y_raw-y_pred)))

In [36]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

In [71]:
def Model_TripleTest(x1, x2, x3, x4, y, vx1, vx2, vx3, vx4):
    xgb1 = BaggingRegressor(XGBRegressor())
    xgb1.fit(x1, y)
    xgb2 = BaggingRegressor(XGBRegressor())
    xgb2.fit(x2, y)
    xgb3 = BaggingRegressor(XGBRegressor())
    xgb3.fit(x3, y)
    xgb4 = BaggingRegressor(XGBRegressor())
    xgb4.fit(x4, y)
    yp1 = xgb1.predict(vx1)
    yp2 = xgb2.predict(vx2)
    yp3 = xgb3.predict(vx3)
    yp4 = xgb4.predict(vx4)
    #return (yp1+yp2+yp3)/3
    xx = pd.concat([x1, x2, x3, x4], axis=1).values
    vxx = pd.concat([vx1, vx2, vx3, vx4], axis=1).values
    xgb5 = BaggingRegressor(XGBRegressor())
    xgb5.fit(xx, y)
    yp5 = xgb4.predict(vxx)
    return (yp1+yp2+yp3+yp4+yp5)/5

In [74]:
x1, vx1, y1, vy1 = train_test_split(dfx_float[:500], train_y, test_size=0.2, random_state=20)
x2, vx2, y2, vy2 = train_test_split(dfx_log1p[:500], train_y, test_size=0.2, random_state=20)
x3, vx3, y3, vy3 = train_test_split(dfx_exp[:500], train_y, test_size=0.2, random_state=20)
x4, vx4, y4, vy4 = train_test_split(dfx_onehot[:500], train_y, test_size=0.2, random_state=20)
print(np.sum(y1 != y2))
print(np.sum(y1 != y3))
yp = Model_TripleTest(x1, x2, x3, x4, y1, vx1, vx2, vx3, vx4)
print(MSE(vy1, yp))

0
0
0.023899030814285425
None


In [80]:
test1_pred = Model_TripleTest(dfx_float[:500], dfx_log1p[:500], dfx_exp[:500], dfx_onehot[:500], train_y, dfx_float[500:600], dfx_log1p[500:600], dfx_exp[500:600], dfx_onehot[500:600])
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_noSynchronous_merge5_20180104.csv', index=False, header=False)

In [81]:
x1, vx1, y1, vy1 = train_test_split(xfloatlr[:500], train_y, test_size=0.2, random_state=20)
x2, vx2, y2, vy2 = train_test_split(xlog1plr[:500], train_y, test_size=0.2, random_state=20)
x3, vx3, y3, vy3 = train_test_split(xexplr[:500], train_y, test_size=0.2, random_state=20)
x4, vx4, y4, vy4 = train_test_split(xonehot[:500], train_y, test_size=0.2, random_state=20)
print(np.sum(y1 != y2))
print(np.sum(y1 != y3))
yp = Model_TripleTest(x1, x2, x3, x4, y1, vx1, vx2, vx3, vx4)
print(MSE(vy1, yp))

0
0
0.0252629350282027
None


In [39]:
lr = XGBRegressor()
#test1_pred = Model_TripleTest(xfloatlr[:500], xlog1plr[:500], xexplr[:500], xonehot[:500], train_y, xfloatlr[500:600], xlog1plr[500:600], xexplr[500:600], xonehot[500:600])
lr.fit(dfx_log1p[:500], train_y)
test1_pred = lr.predict(dfx_log1p[500:600])
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_noSynchronous_log_xgb_20180108.csv', index=False, header=False)